<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1105Sarsa_Q_learning_expected_sarsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TD (時間差)学習, SARSA, 期待 SARSA, Q 学習 と Python 実装

<!-- date: 2020-0519-->
- title: Reinforcement learning: Temporal-Difference, SARSA, Q-Learning & Expected SARSA in python
- author: Vaibhav Kumar
- Date: May 9, 2019
- Original: <https://towardsdatascience.com/reinforcement-learning-temporal-difference-sarsa-q-learning-expected-sarsa-on-python-9fecfda7467e>

<!-- 
# TD, SARSA, Q-Learning and Expected SARSA along with their python implementation and comparison
-->

<!--
> If one had to identify one idea as central and novel to reinforcement learning, it would undoubtedly be temporal-diff
erence (TD) learning. — Andrew Barto and Richard S. Sutton
-->

> 強化学習の中心的で斬新なアイデアを一つ挙げるとすれば、それは間違いなく時間差（TD）学習であろう。<br/>
>    -- アンドリュー・バルト， リチャード・S・サットン

<!-- # Pre-requisites
- Basics of Reinforcement learning
- Markov chains, Markov Decision Process (MDPs)
- Bellman equation
- Value, policy functions and iterations
-->


In [1]:
from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

if isColab:
    !sudo apt-get update
    !sudo apt-get dist-upgrade
    !sudo apt install xorg-dev libx11-dev libgl1-mesa-glx
    !pip install gym[toy_text]

In [2]:
# import pygame
# from pygame.locals import *
# import os
# #os.environ["SDL_VIDEODRIVER"] = "dummy"
# pygame.init()

## 0.1. 前提知識

- 強化学習の基本
- マルコフ連鎖, マルコフ決定プロセス(MDP)
- ベルマン方程式
- 価値，方策関数，反復

<!-- # Model-dependent and model-free reinforcement learning
Model-dependent RL algorithms (namely value and policy iterations) work with the help of a transition table. A transiti
on table can be thought of as a life hack book which has all the knowledge the agent needs to be successful in the worl
d it exists in. Naturally, writing such a book is very tedious and impossible in most cases which is why model dependen
t learning algorithms have little practical use.

Temporal Difference is a model-free reinforcement learning algorithm. This means that the agent learns through actual e
xperience rather than through a readily available all-knowing-hack-book (transition table). This enables us to introduc
e stochastic elements and large sequences of state-action pairs. The agent has no idea about the reward and transition 
systems. It does not know what will happen on taking an arbitrary action at an arbitrary state. The agent has to intera
ct with “world” or “environment” and find out for itself.-->


## 0.2 強化学習におけるモデル依存学習とモデル自由学習

モデル依存 強化学習 (RL) アルゴリズム（すなわち，価値と方策反復）は，遷移表に基づいて動作する。
遷移表は，動作主 (エージェント) が存在する世界で成功するために必要なすべての知識が書かれたライフハック本と考えることができます。
当然ながら，そのような本を書くのは非常に面倒で，ほとんどの場合不可能です。

TD (Temporal Difference) はモデルフリー型の強化学習アルゴリズムである。
これは，動作主 (エージェント) がすぐに入手可能な全知全能のハックブック (遷移表) ではなく，実際の経験を通して学習することを意味する。
これにより，確率的な要素と状態と行動との対の大規模な系列を導入することが可能となる。
動作主 (エージェント) は，報酬システムと遷移システムについては何も知りません。
動作主 (エージェント) は，任意の状態で任意の行動をとった場合に何が起こるかを知らない。
動作主 (エージェント) は，「世界」や「環境」と相互作用し、自分自身で見つけ出さなければなりません。


## 0.3 TD 学習 (時差学習)

TD 時間差学習アルゴリズムは、エージェントが取る一つ一つの行動を通して学習することを可能にします。
TD 学習では、エピソード（ゴールや終了状態に到達する）ごとではなく、タイムステップ（行動）ごとにエージェントの知識を更新します。

$$
\text{新しい状態評価} \leftarrow \text{古い状態評価} + \text{ステップサイズ}\left[\text{目標} - \text{古い状態評価}\right]
$$

<!-- The value Target-OldEstimate is called the target error. StepSize is usually denoted by α is also called the learn
ing rate. Its value lies between 0 and 1.

The equation above helps us achieve **Target** by making updates at every timestep. 
Target is the utility of a state. 
Higher utility means a better state for the agent to transition into. For the sake of brevity of this post, I have assu
med the readers know about the [Bellman equation](https://en.wikipedia.org/wiki/Bellman_equation). According to it, the
 utility of a state is the expected value of the discounted reward as follows:-->

上式の 目標-古い状態評価 を目標誤差と呼びます。
ステップサイズ は 通常 $\alpha$ で表され、学習率と呼ばれます。$\alpha$ の値は 0 から 1 の間になります。

上式は、タイムステップごとに更新を行うことで、**Target** を達成するのに役立ちます。
目標とは状態の効用(関数)です。
効用が高ければ、エージェントが移行しやすい状態になることを意味します。
ここでは [Bellman 方程式](https://en.wikipedia.org/wiki/Bellman_equation) を知っている と仮定しています。
ベルマン方程式から，ある状態の効用は、以下の割引報酬の期待値です。 

$$
\text{目標}=\mathbb{E}_{\pi}\left[\sum_{k=0}^{\infty}\gamma^k r_{t+k+1}\right]
$$

<!-- In layman terms, we are letting an agent run free into a world. The agent has no knowledge of the state, the rewar
ds and transitions. It interacts with the environment (make random or informed actions) and learns new estimates (value
s of state-action pairs) by updating it’s existing knowledge continuously after taking every action.

The discussion till now shall give rise to several questions such as — What is an environment? How will the agent inter
act with the environment? How will the agent choose actions i.e what action will the agent take in a particular state (
policy)?

This is where SARSA and Q-Learning come in. These are the two control policies that will guide our agent in an environm
ent and enable it to learn interesting things. But before that, we shall discuss what is the environment.-->


平たく言えば、我々はエージェントを世界に自由に走らせます。
エージェントは、状態、報酬、遷移についての知識を持っていません。
エージェントは環境と相互作用し（ランダムな行動や情報に基づいた行動をとる）、すべての行動をとった後に既存の知識を継続的
に更新することで、 新たな推定値（状態と行動のペアの値）を学習します。

これまでの議論では、次のようないくつかの疑問が出てきます。
エージェントはどのように環境と相互作用するのか？
エージェントはどのように行動を選択するのか、すなわち 特定の状態（ポリシー） でどのような行動をとるのか？

これが SARSA と Q-学習 の出番です。
これらは、環境の中でエージェントを誘導し、興味深いことを学ぶことを可能にする ２ つの 制御ポリシーです。
これらを説明する前に、環境とは何かを議論しなければなりません。


# 環境
<!--
An environment can be thought of as a mini-world where an agent can observe discrete states, take actions and observe r
ewards by taking those actions. Think of a video game as an environment and yourself as the agent. In the game Doom, yo
u as an agent will observe the states (screen frames) and take actions (press keys like Forward, backward, jump, shoot 
etc) and observe rewards. Killing an enemy would yield you pleasure (utility) and a positive reward while moving ahead 
won’t yield you much reward but you would still want to do that to get future rewards (find and then kill the enemy). C
reating such environments can be tedious and hard ([a team of 7 people worked for more than a year to develop Doom](htt
ps://en.wikipedia.org/wiki/Development_of_Doom)).-->

環境は、エージェントが離散的な状態を観察し、行動をとり、その行動をとることで報酬を観察することができるミニ世界と考える
ことができます。
ビデオゲームは環境であり、自分自身がエージェントであると考えてください。
ゲーム「ドゥーム」では、エージェントであるあなたは、状態（画面のフレーム）を観察し、アクション（前進、後退、ジャンプ、
シュートなどのキーを押す）を行い、報酬を観察します。
敵を殺せば喜び (効用) が得られ、前進している間はプラスの報酬が得られ、あまり報酬は得られませんが、将来の報酬(敵を見つけ
て殺す) を得るために ゲームをしたいと思うでしょう。
このような環境を作るのは面倒で大変な作業です ([7人のチームが1年以上かけて Doom を開発](https://en.wikipedia.org/wiki/Development_of_Doom))。

<!-- 
OpenAI gym comes to the rescue! gym is a python library that has several in-built environments on which you can test va
rious reinforcement learning algorithms. It has established itself as an academic standard to share, analyze and compar
e results. Gym is very well [documented](https://gym.openai.com/docs/) and super easy to use. You must read the documen
ts and familiarize yourself with it before proceeding further.

For novel applications of reinforcement learning, you will have to create your own environments. It’s advised to always
 refer and write gym compatible environments and release them publicly so that everyone can use them. Reading the gym’s
 source code will help you do that. It is tedious but fun!-->


OpenAI gym が登場したことは福音です。
gym は 様々な強化学習アルゴリズムをテストできる環境が組み込まれている Python ライブラリ です。
結果を共有したり、分析したり、比較したりするための学術的な標準環境としての地位を確立しています。
Gym は [ドキュメント](https://gym.openai.com/docs/)が整備されていて、使いやすいです。
ドキュメントを読んで慣れておく必要があります。

強化学習の応用のためには、自分で環境を作る必要があります。
常に gym 互換の環境を参考にして書いて、誰もが使えるように公開しておくことをお勧めします。
Gym の ソースコードを読めば、公開ができるようになります。
面倒だけど楽しい！ と思っている人にはおすすめです。


# 1. SARSA
<!-- > SARSA is acronym for State-Action-Reward-State-Action-->

> SARSA とは，State-Action-Reward-State-Action の省略形です

<!-- SARSA is an on-policy TD control method. A policy is a state-action pair tuple. In python, you can think of it as 
a dictionary with keys as the state and values as the action. Policy maps the action to be taken at each state. An on-p
olicy control method chooses the action for each state during learning by following a certain policy (mostly the one it
 is evaluating itself, like in policy iteration). Our aim is to estimate $QQ\pi(s,a)$ for the current policy $\pi$ and 
all state-action $(s-a)$ pairs. We do this using TD update rule applied at every timestep by letting the agent transiti
on from one state-action pair to another state-action pair (unlike model dependent RL techniques where the agent transi
tions from a state to another state).-->

SARSA とははオンポリシーな 時間差制御方式 です。
ポリシー (方策) とは 状態 と 動作(行動) とのペアのことです。
python では 状態 を キー、動作(行動) を 値 とする 辞書 dict と考えることができます。
ポリシー(方策) は各状態 で取るべき 動作（行動） をマッピングします。
オンポリシー制御では、学習中にある ポリシー (大抵はポリシー反復 のように自分自身で評価しているもの) に従うことで、 状態
ごとに 動作 (行動) を選択します。
我々の目的は、現在の 方策 $\pi$ と全ての 状態行動 $(s-a)$ のペアについて、 $Q \pi(s,a)$ を推定することです。
これは、ある 状態-動作(行動) の対 から 別の 状態-動作(行動)  のペア に エージェント を遷移させることで、 タイムステップ
 ごとに 適用される TD 更新規則 を用いて行う (状態 から 別の 状態に エージェントを遷移させる モデル依存型 強化学習技法と
は異なります)。 

<!-- **Q-value** You must be already familiar with the utility value of a state, Q-value is the same with the only diff
erence of being defined over the state-action pair rather than just the state. It’s a mapping between state-action pair
 and a real number denoting its utility. Q-learning and SARSA are both policy control methods which work on evaluating 
the optimal Q-value for all action-state pairs.-->

**Q-値** 状態の効用値についてはすでにお馴染みでしょうが、Q-値 も同じです。
Q-値 は、状態と行動のペアとその効用を表す実数とのマッピングです。
Q-学習 と SARSA とは、すべての 行動-状態 のペアに対して 最適な Q-値 を評価する 方策制御手法です。

<!-- The update rule for SARSA is:-->

SARSA の更新則は:

$$
Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha\left[ R_{t+1}+\gamma Q(S_{t+1},A_{t+1}-Q(S_t,A_t)\right]
$$
<!--![Source: Introduction to Reinforcement learning by Sutton and Barto — 6.7](https://cdn-images-1.medium.com/max/1280/1*3Ul422CbPyIDJI0XJVun3Q.png) -->

<!-- If a state $S$ is terminal (goal state or end state) then, $Q(S,a)=0\forall a\in A$ where A is the set of all possible 
actions-->

ある状態 $S$ が終了した場合 (ゴールに達したり，終了状態に陥った場合)， $Q(S,a)=0\forall a\in A$ となります。
ここで，$A$ は全行動レパートリーを表します。

<center>
<img src="https://cdn-images-1.medium.com/max/1280/1*fYuXsaJoyCWuIZu49vx_GA.png" width="49%"><br/>
Source: Introduction to Reinforcement learning by Sutton and Barto —Chapter 6
</center>

<center>
<div style="color:white;background-color:gray;width:49%;">
SARSA (オンポリシー TD 制御) $Q\approx q_{\star}$ を推定</div>
<div style="background-color:whitesmoke;width:49%;text-align:left">

- $Q(s,a)$ を全状態 $s$ と 全動作 $a$ について初期化，$Q(\text{終了状態},\cdot)=0$ とする
- 各エピソードを繰り返す:
    - $S$ を初期化する
    - $S$ の中から Q 関数に従って $A$ を選ぶ
    - $Q(S,A)\leftarrow Q(S,A)+\alpha\left[R+\gamma Q(S',S')-Q(S,A)\right]$
    - $S\leftarrow S'$, $A\leftarrow A'$ 
- $S$ が収束するまで繰り返す
</div></center>

## イプシロン ($\epsilon$) 貪欲(欲張り) 方策
<!-- Epsilon-greedy policy is this: -->
イプシロン貪欲な方策とは以下のことを言います:

1. 0 から 1 の範囲 ($r\in[0,1]$) の乱数 $r$ を一つ発生させます
2. もし $r>\epsilon$ であれあば，ランダムにある行動を選択します
3. そうでなければ (すなわち $r\le\epsilon$) $Q$ 値 (最大効用をあたえる) 行動を選択します

<!-- 
1. Generate a random number $r\in[0,1]$
2. If $r>\epsilon$ choose a random action
3. Else choose an action derived from the $Q$ values (which yields the maximum utility)-->

<!-- It shall become more clear after reading the python code.-->

以下に Python コードを示します:
<!--It shall become more clear after reading the python code.-->

In [3]:
def epsilon_greedy(Q,         # 状態 X 行動 から 価値 を与える
                   epsilon,   # 新規行動の探索率 イプシロン貪欲戦略のために使用
                   n_actions, # 可能な行動の選択肢数
                   s,         # 状態数
                   train=False, # `True` Q 値を最大にする行動が決定論的に選択される
                  ):
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

<!--
source: https://gist.githubusercontent.com/TimeTraveller-San/40a7a2655743bf6230706d45d1201b49/raw/25ddaaf54e946d33576e27e6bab45a40f22864a9/epsilon_greedy.py -->

$\epsilon$ の値は **探索 = 知識利用 のジレンマ** を決定します。英語では exploration-exploitatoin dilemma と呼びます。綴が似ているために英語で覚えた方が良いでしょう。

- $\epsilon$ が大きければ、乱数 $r$ が $\epsilon$ より大きくなることはほとんどなく、 ランダムな行動はほとんど起こらない (探索が少なく、知識利用 が多くなる)。
- $\epsilon$ が小さければ、乱数 $r$ は $\epsilon$ よりも大きくなることが多いので、 エージェント は より 多くのランダム な 行動を選択することになる。

このような確率的な性質を利用して、エージェントは環境をより探索することができるようになります。

経験則として、$\epsilon$ は通常 $0.9$ に選ばれます。
ですが $\epsilon$ は 環境タイプに応じて変化させることができます。
いくつかのケースでは、より高い探索に続いてより高い知識利用 を可能にするために、時間の経過とともに $\epsilon$ は 緩和 漸減 されます。

以下 [OpenAI Gym Taxi-v3 環境](https://gym.openai.com/envs/Taxi-v3/) に適用された SARSA のシンプルな Python コードを示します。



## Taxi-v3 環境の説明

* Taxi-v3 課題は [Dietterich(2000)](https://arxiv.org/pdf/cs/9905014.pdf) で階層型強化学習の問題点を説明するために導入されました。
* 4 つの場所（異なる文字でラベル付けされている）があり， エージェントの仕事は， ある場所で乗客を拾い， 別の場所で降ろすことです。
* 送迎が成功すると +20点，時間がかかるごとに 1 点ずつ減ります。
* また，違法な乗降行為には 10 ポイントのペナルティがあります。
<!--  This task was introduced in [Dietterich2000] to illustrate some issues in hierarchical reinforcement learning. 
There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. 
You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions. -->

In [4]:
import gym
env = gym.make('Taxi-v3')
s = env.reset()
print(f'type(s):{type(s)}, len(s):{len(s)}')
print(f's:{s}')

n_states, n_actions = env.observation_space.n, env.action_space.n
print(n_states, n_actions)
print(dir(env))
print(env.action_space)
print(env.observation_space)

type(s):<class 'tuple'>, len(s):2
s:(389, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})
500 6
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_action_space', '_elapsed_steps', '_is_protocol', '_max_episode_steps', '_metadata', '_np_random', '_observation_space', '_reward_range', 'action_space', 'class_name', 'close', 'env', 'metadata', 'np_random', 'observation_space', 'render', 'render_mode', 'reset', 'reward_range', 'spec', 'step', 'unwrapped']
Discrete(6)
Discrete(500)


In [5]:
env = gym.make('Taxi-v3')
n_states, n_actions = env.observation_space.n, env.action_space.n
Q = init_q(n_states, n_actions, type="ones")

s = env.reset()
a = epsilon_greedy(Q, epsilon, n_actions, s[0])

#X = env.step(a)
#s_, reward, done, info = env.step(a)
print(type(X), len(X))
print(X)

s_, reward, done, info, _ = env.step(a)
#s_, reward, done, info = env.step(a)
a_ = epsilon_greedy(Q, epsilon, n_actions, s_)
print(f's:{s},a:{a}, s_:{s_}, a_:{a_}')

NameError: name 'init_q' is not defined

In [6]:
"""
Sutton and  Barto の著書にある SARSA アルゴリズムの python 実装。
SARSA とは ポリシー (方針) 学習の一つ。
SARSA と呼ばれる理由は  (State, Action, Reward, State, Action) だからです。
SARSA と Q 学習 の違いは，SARSA が現在の方針 (ポリシー) に基づいて次の動作を決定するのに対し，Q 学習 は次の状態の効用が最大となる行動を選択すること
"""
import gym
import numpy as np
import time

def init_q(s,           # 状態の数
           a,           # 行動の数
           type="ones"  # `ones` であれば状態遷移表を 1 で初期化，`random` であれば乱数で初期化
                        # `zeros` であれば 0 で初期化
          ):
    """ Q 表の初期化"""
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))

    
def epsilon_greedy(Q,           # 状態 X 行動 から 価値 を与える
                   epsilon,     # 新規行動の探索率 イプシロン貪欲戦略のために使用
                   n_actions,   # 可能な行動の選択肢数
                   s,           # 状態
                   train=False, # `True` Q 値を最大にする行動が決定論的に選択される
                  ):
    """ イプシロン貪欲に行動を探索する"""
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action


def sarsa(alpha,         # 学習率
          gamma,         # 割引率
          epsilon,       # イプシロン貪欲な探索率
          episodes,      # 最大エピソード数
          max_steps,     # 最大ステップ数
          n_tests,       # エピソード数
          render=False, 
          test=False):
    """
    alpha: 学習率
    gamma: 割引率
    epsilon: イプシロン貪欲な探索率
    max_steps: 各エピソードの最大ステップ数
    n_tests: number of test episodes
    """
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        print(f"エピソード: {episode}")
        total_reward = 0
        s = env.reset()
        s = s[0]
        #a = epsilon_greedy(Q, epsilon, n_actions, s[0])
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        done = False
        while t < max_steps:
            if render:
                env.render()
            t += 1
            s_, reward, done, info, _ = env.step(a)
            #s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = epsilon_greedy(Q, epsilon, n_actions, s_)
            if done:
                Q[s, a] += alpha * ( reward  - Q[s, a] )
            else:
                Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_] ) - Q[s, a] )
            s, a = s_, a_
            if done:
                if render:
                    print(f"このエピソードでのステップ数 {t}，総報酬:{total_reward}")
                timestep_reward.append(total_reward)
                break
    if render:
        print(f"Q 値:\n{Q}\nテスト開始:")
    if test:
        test_agent(Q, env, n_tests, n_actions)
    return timestep_reward


def test_agent(Q, env, n_tests, n_actions, delay=0.1):
    for test in range(n_tests):
        print(f"Test #{test}")
        s = env.reset()
        done = False
        epsilon = 0
        total_reward = 0
        while True:
            time.sleep(delay)
            env.render()
            a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
            print(f"状態 {s} で行動 {a} を選択")
            s, reward, done, info = env.step(a)
            total_reward += reward
            if done:
                print(f"エピソード総報酬: {total_reward}")
                time.sleep(1)
                break

In [7]:
%%time
alpha = 0.4
gamma = 0.999
epsilon = 0.9
episodes = 3000
max_steps = 2500
n_tests = 20
timestep_reward = sarsa(alpha, gamma, epsilon, episodes, max_steps, n_tests, test = True)
#timestep_reward = qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, test = True)
print(timestep_reward)

エピソード: 0
エピソード: 1
エピソード: 2
エピソード: 3
エピソード: 4
エピソード: 5
エピソード: 6
エピソード: 7
エピソード: 8
エピソード: 9
エピソード: 10
エピソード: 11
エピソード: 12
エピソード: 13
エピソード: 14
エピソード: 15
エピソード: 16
エピソード: 17
エピソード: 18
エピソード: 19
エピソード: 20
エピソード: 21
エピソード: 22
エピソード: 23
エピソード: 24
エピソード: 25
エピソード: 26
エピソード: 27
エピソード: 28
エピソード: 29
エピソード: 30
エピソード: 31
エピソード: 32
エピソード: 33
エピソード: 34
エピソード: 35
エピソード: 36
エピソード: 37
エピソード: 38
エピソード: 39
エピソード: 40
エピソード: 41
エピソード: 42
エピソード: 43
エピソード: 44
エピソード: 45
エピソード: 46
エピソード: 47
エピソード: 48
エピソード: 49
エピソード: 50
エピソード: 51
エピソード: 52
エピソード: 53
エピソード: 54
エピソード: 55
エピソード: 56
エピソード: 57
エピソード: 58
エピソード: 59
エピソード: 60
エピソード: 61
エピソード: 62
エピソード: 63
エピソード: 64
エピソード: 65
エピソード: 66
エピソード: 67
エピソード: 68
エピソード: 69
エピソード: 70
エピソード: 71
エピソード: 72
エピソード: 73
エピソード: 74
エピソード: 75
エピソード: 76
エピソード: 77
エピソード: 78
エピソード: 79
エピソード: 80
エピソード: 81
エピソード: 82
エピソード: 83
エピソード: 84
エピソード: 85
エピソード: 86
エピソード: 87
エピソード: 88
エピソード: 89
エピソード: 90
エピソード: 91
エピソード: 92
エピソード: 93
エピソード: 94
エピソード: 95
エピソード: 96
エピソード: 97
エピソード: 98
エピソード: 99
エピソード: 100

/opt/anaconda3/lib/python3.9/site-packages/gym/envs/toy_text/taxi.py:282: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("Taxi-v3", render_mode="rgb_array")
  logger.warn(


AssertionError: Something went wrong with pygame. This should never happen.

# 3. Q 学習

Q-学習 は、方策によらない TD 制御です。
SARSA とほぼ同じです。唯一の違いは、次の 動作(行動) $A$ を見つけるための 方策に従うのではなく、 貪欲に 動作(行動)  を選
択することです。 
SARSA と同様に Q値 を評価することを目的としており、更新則 は次のようになります:

$$
Q(S_t,A_t)\leftarrow Q(S_t,A_t)+\alpha\left[R_{t+1}+\gamma\max_{\alpha} Q(S_{t+1},a)-Q(S_t,A_t)\right]
$$

SARSA では、ある方策 に従って 行動 $A'$ を選択していました。
これに対し、上式 Q 学習 では 行動 $A'$ ($a$) は、 $Q$ の最大値を取るだけの 欲張り(貪欲, グリーディ)な 方法で選択されま
す。

Q 学習のアルゴリズムを以下に示します:

<center>
<img src="https://cdn-images-1.medium.com/max/1280/1*Rf_H0YXhSPPm-iyBY2Gnjg.png" width="49%"><br/>

Source: [Introduction to Reinforcement learning by Sutton and Barto — Chapter 6](https://cdn-images-1.medium.com/max/1280/1*Rf_H0YXhSPPm-iyBY2Gnjg.png)
</center>

<center>
<!-- <div style="background-color:lightgray;width:49%;text-align:left"> -->
<div style="color:white;background-color:gray;width:49%;align:center">

Q 学習 (オフポリシー TD 制御) for $\pi\approx\pi_{\star}$
</div>
<div style="background-color:whitesmoke;width:49%;text-align:left">

- $Q(s,a)$ を全状態 $s$ と 全動作 $a$ について初期化，$Q(\text{終了状態},\cdot)=0$ とする
- 各エピソードを繰り返す:
    - $S$ を初期化する
    - 各エピソードを繰り返し
        - $S$ の中から Q 関数に従って $A$ を選ぶ
        - 行動 $A$ を起こし，$R$, $S'$ を観察する
        - $Q(S,A)\leftarrow Q(S,A)+\alpha\left[R+\gamma \max_{a} Q(S',a)-Q(S,A)\right]$
        - $S\leftarrow S'$ 
</div></center>

Python コードは以下になります:

In [ ]:
import gym
import numpy as np
import time

"""
Q 学習はオフポリシー学習のpython実装です。
Q 学習はオフポリシー学習のpython実装です。
これは Sutton and Barto の著書 にある Q 学習アルゴリズムの python 実装です。
SARSA と Q 学習の唯一の違いは、SARSA は現在のポリシーに基づいて次の行動を取るのに対し 現在の方針に基づいて次の行動をとるのに対し、Q 学習は次の状態で最大の効用を持つ行動をとることです。行動をとることです。
"""

def init_q(s, a, type="ones"):
    """
    @param s the number of states
    @param a the number of actions
    @param type random, ones or zeros for the initialization
    """
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))


def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
    """
    @param Q Q values state x action -> value
    @param epsilon for exploration
    @param s number of states
    @param train if true then no random actions selected
    """
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

def qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
    """
    @param alpha learning rate
    @param gamma decay factor
    @param epsilon for exploration
    @param max_steps for max step in each episode
    @param n_tests number of test episodes
    """
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        print(f"Episode: {episode}")
        s = env.reset()
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        total_reward = 0
        done = False
        while t < max_steps:
            if render:
                env.render()
            t += 1
            s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = np.argmax(Q[s_, :])
            if done:
                Q[s, a] += alpha * ( reward  - Q[s, a] )
            else:
                Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_]) - Q[s, a] )
            s, a = s_, a_
            if done:
                if render:
                    print(f"This episode took {t} timesteps and reward: {total_reward}")
                timestep_reward.append(total_reward)
                break
    if render:
        print(f"Here are the Q values:\n{Q}\nTesting now:")
    if test:
        test_agent(Q, env, n_tests, n_actions)
    return timestep_reward

def test_agent(Q, env, n_tests, n_actions, delay=1):
    for test in range(n_tests):
        print(f"Test #{test}")
        s = env.reset()
        done = False
        epsilon = 0
        while True:
            time.sleep(delay)
            env.render()
            a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
            print(f"Chose action {a} for state {s}")
            s, reward, done, info = env.step(a)
            if done:
                if reward > 0:
                    print("Reached goal!")
                else:
                    print("Shit! dead x_x")
                time.sleep(3)
                break


In [ ]:
%%time
alpha = 0.4
gamma = 0.999
epsilon = 0.9
episodes = 10000
max_steps = 2500
n_tests = 2
timestep_reward = qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, test = True)
print(timestep_reward)

source: https://gist.github.com/TimeTraveller-San/9e56f9d09be7d50b795ef2f83be2ba72#file-qlearning-py

# 3. 期待 SARSA

<!-- Expected SARSA, as the name suggest takes the expectation (mean) of Q values for every possible action in the curr
ent state. The target update rule shall make things more clear:-->

期待 SARSA とは、その名が示すように、現在の状態で起こりうるすべての行動についての Q値 の期待値(平均値)を取ります。
ターゲットの更新則を使うと、より明確になります。

$$
Q(S_t,A_t)\leftarrow Q(S_t,A_t) + \alpha\left[R_{t+1}+\gamma\mathbb{E}\left[Q(S_{t+1},A_{t+1}\vert S_{t+1}\right]-Q(S_t
,A_t)\right]\\ \\
\hspace{6em}\leftarrow Q(S_t,A_t)+\alpha\left[
R_{t+1}+\gamma\sum_{\alpha}\pi(a\vert S_{t+1}) Q(S_{t+1},a) - Q(S_t,A_t)
\right]
$$

Python コードを以下に示します:

In [ ]:
import gym
import numpy as np
import time


def init_q(s, a, type="ones"):
    """
    @param s the number of states
    @param a the number of actions
    @param type random, ones or zeros for the initialization
    """
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))


def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
    """
    @param Q Q values state x action -> value
    @param epsilon for exploration
    @param s number of states
    @param train if true then no random actions selected
    """
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

def expected_sarsa(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
    """
    @param alpha learning rate
    @param gamma decay factor
    @param epsilon for exploration
    @param max_steps for max step in each episode
    @param n_tests number of test episodes
    """
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        print(f"Episode: {episode}")
        total_reward = 0
        s = env.reset()
        t = 0
        done = False
        while t < max_steps:
            if render:
                env.render()
            t += 1
            a = epsilon_greedy(Q, epsilon, n_actions, s)
            s_, reward, done, info = env.step(a)
            total_reward += reward
            if done:
                Q[s, a] += alpha * ( reward  - Q[s, a] )
            else:
                expected_value = np.mean(Q[s_,:])
                # print(Q[s,:], sum(Q[s,:]), len(Q[s,:]), expected_value)
                Q[s, a] += alpha * (reward + (gamma * expected_value) - Q[s, a])
            s = s_
            if done:
                if True:
                    print(f"This episode took {t} timesteps and reward {total_reward}")
                timestep_reward.append(total_reward)
                break
    if render:
        print(f"Here are the Q values:\n{Q}\nTesting now:")
    if test:
        test_agent(Q, env, n_tests, n_actions)
    return timestep_reward

def test_agent(Q, env, n_tests, n_actions, delay=0.1):
    for test in range(n_tests):
        print(f"Test #{test}")
        s = env.reset()
        done = False
        epsilon = 0
        total_reward = 0
        while True:
            time.sleep(delay)
            env.render()
            a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
            print(f"Chose action {a} for state {s}")
            s, reward, done, info = env.step(a)
            total_reward += reward
            if done:
                print(f"Episode reward: {total_reward}")
                time.sleep(1)
                break


In [ ]:
%%time
alpha = 0.1
gamma = 0.9
epsilon = 0.9
episodes = 1000
max_steps = 2500
n_tests = 20
timestep_reward = expected_sarsa(alpha, gamma, epsilon,
                                 episodes, max_steps, n_tests,
                                 render=False, test=True
                                 )
print(timestep_reward)


source: https://gist.github.com/TimeTraveller-San/5bd93710e118633e0793dc5d0b92b19a#file-expectedsarsa-py

# 比較

以下のパラメータで 3 つのアルゴリズムを比較しました

```python
lpha = 0.4
gamma = 0.999
epsilon = 0.9
episodes = 2000
max_steps = 2500 # (max number of time steps possible in a single episode)
```

ここでは、上記の 3 つの方策制御方法の比較 をプロットします。

## 収束

以下のプロットが示すように、Q 学習 (緑) は SARSA(オレンジ) と 期待 SARSA(青) の両者より先に収束しました。

<center>
<img src="https://cdn-images-1.medium.com/max/1280/1*0LpFOud2FUKciZ9jPd1ZBA.png" width="49%"><br/>
SARSA, Q-learning & Expected SARSA — Convergence comparison
</center>

## 成績

実装した 3 つのアルゴリズムでは、Q-学習 が最も良く、期待 SARSA が最も悪い性能のようです。

<center>
<img src="https://cdn-images-1.medium.com/max/1280/1*IC0L25IzedYZ_TujMHpRKg.png"><br/>
SARSA, Q-learning & Expected SARSA — performance comparison!
</center>

# 結語

<!-- Temporal Difference learning is the most important reinforcement learning concept. It’s further derivatives like D
QN and double DQN (I may discuss them later in another post) have achieved groundbreaking results renowned in the field
 of AI. Google’s alpha go used DQN algorithm along with CNNs to defeat the go world champion. You are now equipped with
 the theoretical and practical knowledge of basic TD, go out and explore!
 -->

時間差学習 は 最も重要な強化学習の概念です。 DQN や ダブルDQN のような、さらに派生したもの
は、AI の分野で有名な画期的な結果を達成しています。
Google の アルファ碁は、囲碁の世界チャンピオンを倒すために、CNN と DQN アルゴリズムを使用しました。
